<a href="https://colab.research.google.com/github/eliasingi12/DEEP_U-net_Project/blob/master/DEEP_run_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# get drive database
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import os  # misc operating system specific operations, e.g., reading directries.
import sys
import random

import cv2
import numpy as np

import matplotlib.pyplot as plt

#from unet import unet

#resize inputs & targets
import skimage
from skimage.transform import resize

from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Cropping2D
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.optimizers import SGD
from sklearn.metrics import classification_report

!pip install fcn==6.4.4

Using TensorFlow backend.


    100% |████████████████████████████████| 225kB 21.9MB/s 
  Running setup.py bdist_wheel for fcn ... - \ done
  Stored in directory: /root/.cache/pip/wheels/3e/70/80/a3127881a5d4288bd996131df5a60dfcf16199a235ee60afe9
  Running setup.py bdist_wheel for chainercv ... - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/95/24/5a/1611db416857b5e092962bc22a70722315881e03970c7fa966
Successfully built fcn chainercv


In [0]:
# Some parameters and paths to data
#dir_path = os.path.dirname(os.path.realpath(__file__))
path_train = 'DRIVE/training/'
path_test = 'DRIVE/test/'
path_img = 'images'
path_mask = 'mask'
path_targets = '1st_manual'
path_tif_targets = '1st_manual_tif'
pre_dir = "/content/drive/My Drive/DRIVE/"

In [4]:
# Read in the file paths of the images to use for the training.
random_seed = 42
sz = 64
image_paths = []
target_paths = []

args = {}
args["training_imgs"] = pre_dir + "training/images/"
args["targets"] = pre_dir + "training/1st_manual_tif/"

for (dirpath, dirnames, filenames) in os.walk(args["training_imgs"]):
    for file in filenames:
        if '.tif' in file and not file.startswith('.'):
              image_paths.append(os.path.join(dirpath, file))

for (dirpath, dirnames, filenames) in os.walk(args["targets"]):
    for file in filenames:
        if '.tiff' in file and not file.startswith('.'):
              target_paths.append(os.path.join(dirpath, file))
            
                
random.seed(random_seed)

def show_images(imgs, grid_size=3):
    f, axarr = plt.subplots(grid_size,grid_size, figsize=(15, 15))
    for i in range(grid_size):
        for j in range(grid_size):
            axarr[i,j].imshow(imgs[i*grid_size+j])
    plt.show()
  

input_data = []
for image_path in image_paths:
    image = cv2.imread(image_path)
    image = resize(image, (560, 560)) #, anti_aliasing=True)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    input_data.append(image)
    

target_data = []
for target_path in target_paths:
    target = cv2.imread(target_path)
    target = resize(target, (560, 560)) #, anti_aliasing=True)
    #target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    #print(target)
    target_data.append(target)
    #print(target_data)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
def unet(height,width,n_ch):
    inputs = Input((height,width,n_ch))

    # First set of layers
    conv1 = Conv2D(64, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(inputs)
    conv1 = Conv2D(64, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv1)
    pool1 = MaxPooling2D((2,2))(conv1)

    # Second set of layers
    conv2 = Conv2D(128, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(pool1)
    conv2 = Conv2D(128, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv2)
    pool2 = MaxPooling2D((2,2))(conv2)

    # Third set of layers
    conv3 = Conv2D(256, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(pool2)
    conv3 = Conv2D(256, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv3)
    pool3 = MaxPooling2D((2,2))(conv3)

    # Fourth set of layers
    conv4 = Conv2D(512, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(pool3)
    conv4 = Conv2D(512, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv4)
    pool4 = MaxPooling2D((2,2))(conv4)

    # Fifth set of layers
    conv5 = Conv2D(1024, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(pool4)
    conv5 = Conv2D(1024, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv5)

    # First up layers
    upsamp1 = UpSampling2D((2,2))(conv5)
    crop1 = Cropping2D(cropping=((0,0),(0,0)))(conv4)
    concat1 = concatenate([upsamp1,crop1])

    conv6 = Conv2D(512, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(concat1)
    conv6 = Conv2D(512, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv6)

    # Second up layers
    upsamp2 = UpSampling2D((2,2))(conv6)
    crop2 = Cropping2D(cropping=((0,0),(0,0)))(conv3)
    concat2 = concatenate([upsamp2,crop2])

    conv7 = Conv2D(256, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(concat2)
    conv7 = Conv2D(256, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv7)

    # Third up layers
    upsamp3 = UpSampling2D((2,2))(conv7)
    crop3 = Cropping2D(cropping=((0,0),(0,0)))(conv2)
    concat3 = concatenate([upsamp3,crop3])

    conv8 = Conv2D(128, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(concat3)
    conv8 = Conv2D(128, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv8)

    # Fourth up layers
    upsamp4 = UpSampling2D((2,2))(conv8)
    crop4 = Cropping2D(cropping=((0,0),(0,0)))(conv1)
    concat4 = concatenate([upsamp4,crop4])

    conv9 = Conv2D(64, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(concat4)
    conv9 = Conv2D(64, (3,3), padding='same', kernel_initializer='random_uniform', activation='relu', data_format='channels_last')(conv9)

    # Output layer
    outconv = Conv2D(3, (1,1), kernel_initializer='random_uniform', padding='same', activation='sigmoid', data_format='channels_last')(conv9)

    model = Model(inputs=inputs, outputs=outconv)

    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.3, nesterov=False)
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [6]:
h,w,ch = target_data[0].shape

#print(target_data[0].shape)
#print(input_data[0].shape)
#show_images(target_data)

EPOCHS=5

model = unet(h,w,ch)
model.summary()


input_data = np.array(input_data)
target_data = np.array(target_data)
print(target_data.shape)
print(input_data.shape)
#input_data = np.reshape(input_data, (20,584,565,3))

model.fit(input_data, target_data, epochs=EPOCHS, batch_size=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 560, 560, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 560, 560, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 560, 560, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 280, 280, 64) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (